In [ ]:
# Para execução desse notebook foi utilizado o ambiente GPUsT4 sem alta RAM do colab

In [ ]:
!python --version

Python 3.11.11


In [ ]:
!git clone https://github.com/ismaelclebson/unet.git

Cloning into 'unet'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 18 (delta 4), reused 18 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 13.32 KiB | 2.22 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [ ]:
!pip install -q git+https://github.com/qubvel/segmentation_models.pytorch

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.1 MB/s eta 0:00:00


In [ ]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 95.2 MB/s eta 0:00:00


In [ ]:
# Imports necessários
import matplotlib
import numpy as np
import torch
import rasterio
from einops import __version__ as einops_version
from tqdm import __version__ as tqdm_version
from torchvision import __version__ as torchvision_version
import yaml
import os
import subprocess

# Obter a versão do Lightning
try:
    import lightning
    lightning_version = lightning.__version__
except ImportError:
    lightning_version = "Not Installed"

# Obter a versão do segmentation_models.pytorch
try:
    segmentation_models_version = subprocess.check_output(
        ["pip", "show", "segmentation-models-pytorch"],
        text=True,
    ).split("Version: ")[1].split("\n")[0]
except (IndexError, subprocess.CalledProcessError):
    segmentation_models_version = "Not Installed"

# Criar um dicionário para organizar as versões
versions = {
    "matplotlib": matplotlib.__version__,
    "numpy": np.__version__,
    "torch": torch.__version__,
    "rasterio": rasterio.__version__,
    "einops": einops_version,
    "tqdm": tqdm_version,
    "torchvision": torchvision_version,
    "yaml": yaml.__version__,
    "lightning": lightning_version,
    "segmentation_models.pytorch": segmentation_models_version,
    "os": "Builtin",
}

# Imprimir as versões
for lib, version in versions.items():
    print(f"{lib}: {version}")

In [ ]:
import sys
import warnings

sys.path.append("/content/unet")
warnings.filterwarnings("ignore")

In [ ]:
import os
import rasterio
import numpy as np
import torch
import torch.nn.functional as F
from rasterio.windows import Window
from pathlib import Path
from tqdm import tqdm
from model import UNet


class GeoTIFFPredictor:
    def __init__(self, model, device, window_size=256, overlap=64):
        self.model = model
        self.device = device
        self.window_size = window_size
        self.overlap = overlap
        self.stride = window_size - overlap
        self.ramp = np.linspace(0, 1, overlap//2)

    def get_blend_weights(self, window_height, window_width, y_start, x_start, src_height, src_width):
        """Gera pesos adaptativos considerando bordas da imagem"""
        weights = np.ones((window_height, window_width), dtype=np.float32)

        # Aplica rampa apenas se não estiver na borda correspondente
        # Topo
        if y_start > 0:
            top_ramp = self.ramp[:, np.newaxis]
            weights[:self.overlap//2, :] *= top_ramp[:window_height, :]

        # Base
        if y_start + window_height < src_height:
            bottom_ramp = self.ramp[::-1, np.newaxis]
            weights[-self.overlap//2:, :] *= bottom_ramp[:window_height, :]

        # Esquerda
        if x_start > 0:
            left_ramp = self.ramp[np.newaxis, :]
            weights[:, :self.overlap//2] *= left_ramp[:, :window_width]

        # Direita
        if x_start + window_width < src_width:
            right_ramp = self.ramp[np.newaxis, ::-1]
            weights[:, -self.overlap//2:] *= right_ramp[:, :window_width]

        return torch.from_numpy(weights).to(self.device).cpu()

    def normalize_band(self, band):
        """Normalização individual de cada banda (igual ao dataset)"""
        min_val = band.min()
        max_val = band.max()
        if max_val > min_val:
            return (band - min_val) / (max_val - min_val)
        return np.zeros_like(band, dtype=np.float32)

    def predict_geotiff(self, input_dir, output_dir, return_probs=True):
        """Executa a predição completa em um arquivo GeoTIFF"""
        for filename in os.listdir(input_dir): # Iterate through files in the input directory
            if filename.endswith(".tif"): # Check if the file is a GeoTIFF
                input_path = os.path.join(input_dir, filename) # Create full path to the input file
                output_path = os.path.join(output_dir, filename[:-4] + "_pred.tif") # Create full path to the output file

                with rasterio.open(input_path) as src:
                    full_pred = np.zeros((src.height, src.width), dtype=np.float32)
                    full_count = np.zeros((src.height, src.width), dtype=np.float32)

                    offsets = []
                    # Gera coordenadas com overlap negativo para cobrir bordas
                    for y in range(-self.overlap//2, src.height, self.stride):
                        for x in range(-self.overlap//2, src.width, self.stride):
                            y_start = max(0, y)
                            x_start = max(0, x)
                            y_end = min(src.height, y_start + self.window_size)
                            x_end = min(src.width, x_start + self.window_size)

                            # Apenas adiciona janelas válidas
                            if (y_end - y_start) > self.overlap//2 and (x_end - x_start) > self.overlap//2:
                                offsets.append((y_start, x_start, y_end, x_end))

                    for y_start, x_start, y_end, x_end in tqdm(offsets, desc="Processando janelas"):
                        window = Window(x_start, y_start, x_end - x_start, y_end - y_start)
                        chip = src.read(window=window).astype(np.float32)

                        # Normaliza cada banda
                        for c in range(chip.shape[0]):
                            chip[c] = self.normalize_band(chip[c])

                        # Converte para tensor
                        input_tensor = torch.from_numpy(chip).unsqueeze(0).to(self.device)

                        # Predição
                        with torch.no_grad():
                            output = self.model(input_tensor)
                            pred = torch.sigmoid(output).squeeze().cpu().numpy()

                        # Obtém dimensões reais
                        h, w = pred.shape

                        # Gera pesos adaptativos
                        weights = self.get_blend_weights(
                            window_height=h,
                            window_width=w,
                            y_start=y_start,
                            x_start=x_start,
                            src_height=src.height,
                            src_width=src.width
                        )

                        # Aplica pesos
                        weighted_pred = pred * weights.numpy()

                        # Atualiza acumuladores
                        full_pred[y_start:y_end, x_start:x_end] += weighted_pred
                        full_count[y_start:y_end, x_start:x_end] += weights.numpy()

                    # Calcula média ponderada
                    full_pred = np.divide(full_pred, full_count, where=full_count>0)

                    # Binarização
                    if not return_probs:
                        full_pred = (full_pred > 0.5).astype(np.uint8)

                    # Salva resultado
                    self.save_geotiff(output_path, full_pred, src.profile, return_probs)

    def save_geotiff(self, output_path, data, profile, return_probs):
        """Salva o resultado mantendo a projeção original"""
        profile.update({
            'driver': 'GTiff',
            'height': data.shape[0],
            'width': data.shape[1],
            'count': 1,
            'dtype': 'float32' if return_probs else 'uint8',
            'nodata': None,
            'compress': 'lzw'
        })

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(data, 1)



In [ ]:
checkpoint_path = "/content/drive/MyDrive/unet-agriculture/oil_palm/model-weight/checkpoint_epoch_34_acc_0.7348_loss_0.1361_2loss_ondecay_1e-3-lr_1e-4.pth"

for region in ['PA','BA', 'RR']:
  for year in range(2012, 2013):
    input_path = f"/content/drive/MyDrive/ClayModel_detail/palm_ls/data/predict/dende/{year}/{region}/"
    output_path = f"/content/drive/MyDrive/ClayModel_detail/palm_ls/data/predict/dende/{year}/{region}/predicted/"

    os.makedirs(output_path, exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = UNet(in_channels=6).to(device)
    model.load_state_dict(torch.load(checkpoint_path))
    model.eval()

    predictor = GeoTIFFPredictor(model, device, window_size=256, overlap=64)
    predictor.predict_geotiff(input_path, output_path, return_probs=True)

Processando janelas: 100%|██████████| 9/9 [00:00<00:00, 50.26it/s]


In [ ]:
!sudo apt-get install gdal-bin -y
!sudo add-apt-repository ppa:ubuntugis/ppa -y
!sudo apt-get update -y
!pip install retry rasterio
import rasterio as rio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-gdal python3-numpy
Suggested packages:
  libgdal-grass python-numpy-doc python3-pytest
The following NEW packages will be installed:
  gdal-bin python3-gdal python3-numpy
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 5,055 kB of archives.
After this operation, 25.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 python3-numpy amd64 1:1.21.5-1ubuntu22.04.1 [3,467 kB]
Get:2 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 python3-gdal amd64 3.6.4+dfsg-1~jammy0 [1,027 kB]
Get:3 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 gdal-bin amd64 3.6.4+dfsg-1~jammy0 [561 kB]
Fetched 5,055 kB in 1s (4,430 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed

In [ ]:
#/content/drive/MyDrive/ClayModel_detail/palm_ls/data/predict/dende/2024/PA/predicted/
TARGET = 'dende'

for region in ['PA','BA', 'RR']:
  print(region)
  for year in range(2012, 2013):
    print(year)
    !cd /content/drive/MyDrive/ClayModel_detail/palm_ls/data/predict/{TARGET}/{year}/{region}/predicted && gdalbuildvrt tmp.vrt *.tif -hidenodata -srcnodata 0
    !cd /content/drive/MyDrive/ClayModel_detail/palm_ls/data/predict/{TARGET}/{year}/{region}/predicted && GDAL_CACHEMAX=512 gdal_translate -of GTiff -a_nodata 0 -co COMPRESS=DEFLATE -co NUM_THREADS=ALL_CPUS -co BIGTIFF=YES  tmp.vrt merged_{TARGET}_{region}_{year}_2025.tiff

PA
2012
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 11776, 11776
0...10...20...30...40...50...60...70...80...90...100 - done.
BA
2012
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 2560, 5120
0...10...20...30...40...50...60...70...80...90...100 - done.
RR
2012
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 11264, 12288
0...10...20...30...40...50...60...70...80...90...100 - done.


In [ ]:
import os
import shutil

TARGET = 'dende'

base_path = "/content/drive/MyDrive/ClayModel_detail/palm_ls/data/predict"

for year in range(2012, 2013):
    print(year)
    for region in ['PA', 'BA', 'RR']:
        print(region)

        # Caminho para a pasta de predição
        predicted_path = f"{base_path}/{TARGET}/{year}/{region}/predicted/merged_{TARGET}_{region}_{year}_2025.tiff"

        # Caminho de destino para copiar o arquivo de merge
        destination_path = f"{base_path}/{TARGET}/{year}/merged_{TARGET}_{region}_{year}_2025.tiff"

        # Copiando o arquivo de merge para a pasta do ano correspondente
        if os.path.exists(predicted_path):
            shutil.copy(predicted_path, destination_path)
            print(f"✅ Copiado para {destination_path}")
        else:
            print(f"⚠️ Arquivo {predicted_path} não encontrado. Verifique se o processo de merge ocorreu corretamente.")


2012
PA
✅ Copiado para /content/drive/MyDrive/ClayModel_detail/palm_ls/data/predict/dende/2012/merged_dende_PA_2012_2025.tiff
BA
✅ Copiado para /content/drive/MyDrive/ClayModel_detail/palm_ls/data/predict/dende/2012/merged_dende_BA_2012_2025.tiff
RR
✅ Copiado para /content/drive/MyDrive/ClayModel_detail/palm_ls/data/predict/dende/2012/merged_dende_RR_2012_2025.tiff


In [ ]:
#/content/drive/MyDrive/ClayModel_detail/palm_ls/data/predict/dende/2024/PA/predicted/
TARGET = 'dende'

for year in range(2012, 2013):
    print(year)
    !cd /content/drive/MyDrive/ClayModel_detail/palm_ls/data/predict/{TARGET}/{year} && gdalbuildvrt tmp.vrt *.tiff -hidenodata -srcnodata 0
    !cd /content/drive/MyDrive/ClayModel_detail/palm_ls/data/predict/{TARGET}/{year} && GDAL_CACHEMAX=512 gdal_translate -of GTiff -a_nodata 0 -co COMPRESS=DEFLATE -co NUM_THREADS=ALL_CPUS -co BIGTIFF=YES  tmp.vrt merged_{TARGET}_{year}_2025.tiff

2012
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 86016, 58880
0...10...20...30...40...50...60...70...80...90...100 - done.


In [ ]:
import os
import shutil

TARGET = 'dende'

base_path = "/content/drive/MyDrive/ClayModel_detail/palm_ls/data/predict"

for year in range(2012, 2013):
    print(year)
    #merged_dende_1985_2025.tiff
    ## Caminho para a pasta de predição
    #predicted_path = f"{base_path}/{TARGET}/{year}/{region}/predicted/merged_{TARGET}_{region}_{year}_2025.tiff"
#
    ## Caminho de destino para copiar o arquivo de merge
    #destination_path = f"{base_path}/{TARGET}/{year}/merged_{TARGET}_{region}_{year}_2025.tiff"

    # Caminho para a pasta de predição
    predicted_path = f"{base_path}/{TARGET}/{year}/merged_{TARGET}_{year}_2025.tiff"

    # Caminho de destino para copiar o arquivo de merge
    destination_path = f"{base_path}/{TARGET}/merged_{TARGET}_{year}_2025.tiff"

    # Copiando o arquivo de merge para a pasta do ano correspondente
    if os.path.exists(predicted_path):
        shutil.copy(predicted_path, destination_path)
        print(f"✅ Copiado para {destination_path}")
    else:
        print(f"⚠️ Arquivo {predicted_path} não encontrado. Verifique se o processo de merge ocorreu corretamente.")


2012
✅ Copiado para /content/drive/MyDrive/ClayModel_detail/palm_ls/data/predict/dende/merged_dende_2012_2025.tiff
